In [1]:
import pandas as pd

# Load the dataset
file_path = './data/Data Insights - Synthetic Dataset.xlsx'
df = pd.read_excel(file_path, sheet_name='Data Insights - Synthetic Datas')



In [ ]:
# Function to identify data quality issues
def identify_data_issues(df):
    print("1. Checking for missing values...")
    missing_values = df.isnull().sum()
    print(missing_values[missing_values > 0])

    print("\n2. Checking for duplicate rows...")
    duplicate_rows = df.duplicated().sum()
    print(f"Number of duplicate rows: {duplicate_rows}")

    print("\n3. Checking for data types...")
    print(df.dtypes)

    print("\n4. Checking for columns with unexpected or inconsistent data types...")
    # Looking for columns that should be numeric but have object types
    numeric_columns = df.select_dtypes(include=['object'])
    print(f"Potential issues in numeric columns stored as objects: {numeric_columns.columns.tolist()}")

    print("\n5. Checking for outliers in numeric columns...")
    numeric_data = df.select_dtypes(include=['float64', 'int64'])
    print(numeric_data.describe())

    print("\n6. Summary of column value ranges and potential data entry issues...")
    for col in numeric_data.columns:
        min_val = numeric_data[col].min()
        max_val = numeric_data[col].max()
        print(f"Column {col} - Min: {min_val}, Max: {max_val}")

    print("\n7. Checking for invalid or extreme outliers (e.g., extremely high pharmacy charges)...")
    extreme_outliers = df[(df['PharmacyCharge'] > 1e10) | (df['AccommodationCharge'] > 1e10)]
    print(f"Number of extreme outliers in charges: {extreme_outliers.shape[0]}")

# Running the data quality check
identify_data_issues(df)